In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir("../input"))

In [ ]:
import numpy as np
import keras, random
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
import plotly.graph_objs as go

In [ ]:
x = np.load('../input/Sign-language-digits-dataset/X.npy')
y = np.load('../input/Sign-language-digits-dataset/Y.npy')

x.shape, y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 8)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size = 0.5, random_state = 8)
x_train = x_train[:, :, :, np.newaxis]
x_test = x_test[:, :, :, np.newaxis]
x_valid = x_valid[:, :, :, np.newaxis]
x_train.shape, x_test.shape, x_valid.shape, y_train.shape, y_test.shape, x_valid.shape

In [ ]:
datagen = ImageDataGenerator(rotation_range = 16, width_shift_range = 0.12, height_shift_range = 0.12, zoom_range = 0.12)
datagen.fit(x_train)

In [ ]:
def convnet():
    model = Sequential()
    
    model.add(Conv2D(input_shape = (64, 64, 1), filters = 64, kernel_size = (4, 4), strides = (2)))
    model.add(LeakyReLU(alpha = 0.1))
#     model.add(Activation('relu'))
    
    model.add(Conv2D(filters = 64, kernel_size = (4, 4), strides = (1)))
    model.add(LeakyReLU(alpha = 0.1))
#     model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = 4))
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(10, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adadelta(), metrics = ['accuracy'])
    model.summary()
    
    return model

In [ ]:
model = convnet()

In [ ]:
SVG(model_to_dot(model, show_shapes = True, show_layer_names = True).create(prog = 'dot', format = 'svg'))

In [ ]:
history_training_phase_1 = model.fit(x_train, y_train, batch_size = 32, epochs = 10, validation_data = (x_valid, y_valid))

In [ ]:
data = [
    go.Scatter(x = list(range(len(history_training_phase_1.history['loss']))), y = history_training_phase_1.history['loss'], name = "Loss"),
    go.Scatter(x = list(range(len(history_training_phase_1.history['val_loss']))), y = history_training_phase_1.history['val_loss'], name = "Validation Loss")
]
layout = dict(title = "Loss History in Training Phase 1", xaxis = dict(title = "Epochs"), yaxis = dict(title = "Loss"))
iplot(dict(data = data, layout = layout), filename = "Loss-History-1")

In [ ]:
data = [
    go.Scatter(x = list(range(len(history_training_phase_1.history['acc']))), y = history_training_phase_1.history['acc'], name = "Accuracy"),
    go.Scatter(x = list(range(len(history_training_phase_1.history['val_acc']))), y = history_training_phase_1.history['val_acc'], name = "Validation Accuracy")
]
layout = dict(title = "Accuracy History in Training Phase 1", xaxis = dict(title = "Epochs"), yaxis = dict(title = "Accuracy"))
iplot(dict(data = data, layout = layout), filename = "Accuracy-History-1")

In [ ]:
score = model.evaluate(x_test, y_test)
print('Test Accuracy after Training Phase 1:', score[1] * 100, '%')

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adadelta(), metrics = ['accuracy'])
history_training_phase_2 = model.fit_generator(datagen.flow(x_train, y_train, batch_size = 32),
                                               steps_per_epoch = 64, epochs = 10,
                                               validation_data = (x_valid, y_valid))

In [ ]:
data = [
    go.Scatter(x = list(range(len(history_training_phase_2.history['loss']))), y = history_training_phase_2.history['loss'], name = "Loss"),
    go.Scatter(x = list(range(len(history_training_phase_2.history['val_loss']))), y = history_training_phase_2.history['val_loss'], name = "Validation Loss")
]
layout = dict(title = "Loss History in Training Phase 2", xaxis = dict(title = "Epochs"), yaxis = dict(title = "Loss"))
iplot(dict(data = data, layout = layout), filename = "Loss-History-2")

In [ ]:
data = [
    go.Scatter(x = list(range(len(history_training_phase_2.history['acc']))), y = history_training_phase_2.history['acc'], name = "Accuracy"),
    go.Scatter(x = list(range(len(history_training_phase_2.history['val_acc']))), y = history_training_phase_2.history['val_acc'], name = "Validation Accuracy")
]
layout = dict(title = "Accuracy History in Training Phase 2", xaxis = dict(title = "Epochs"), yaxis = dict(title = "Accuracy"))
iplot(dict(data = data, layout = layout), filename = "Accuracy-History-2")

In [ ]:
score = model.evaluate(x_test, y_test)
print('Test Accuracy after Training Phase 2:', score[1] * 100, '%')

In [ ]:
test_image = random.choice(x_test).reshape(1, 64, 64, 1)
result = model.predict(test_image)
plt.imshow(test_image.reshape(64, 64), cmap='gray')
plt.title(str(result.argmax()))
plt.show()

In [ ]:
model.save('model.h5')
model.save_weights('model_weights.h5')